# MEModel With Synapses Simulation Example with OBI-One Form Logic

This notebook demonstrates how to run a MEModel With Synapses simulation using the OBI-One form-based workflow, similar to the circuit simulation example.

In [ ]:
from entitysdk import Client
from obi_auth import get_token

import obi_one as obi
from entitysdk import Client, ProjectContext


obi_one_api_url = "http://127.0.0.1:8100"

token = get_token(environment="staging")
project_context = ProjectContext(virtual_lab_id=obi.LAB_ID_STAGING_TEST, project_id=obi.PROJECT_ID_STAGING_TEST)
db_client = Client(api_url="https://staging.openbraininstitute.org/api/entitycore", project_context=project_context, token_manager=token)

In [ ]:
entity_ID = "5c43e35b-7e44-4705-a2a1-2e5ac24af536"

In [ ]:
from pathlib import Path
import obi_one as obi

# === Parameters ===
sim_duration = 3000.0

sim_form = obi.MEModelWithSynapsesCircuitSimulationScanConfig.empty_config()

# Info
info = obi.Info(
    campaign_name="MEModel With Synapses Simulation",
    campaign_description="Simulation of example MEModel With Synapses"
)
sim_form.set(info, name="info")


# Timestamps
timestamps = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=1, interval=100)
sim_form.add(timestamps, name="Timestamps")

# Stimulus
stimulus = obi.PoissonSpikeStimulus(
    duration=800.0,
    timestamps=timestamps.ref,
    frequency=20,
)
sim_form.add(stimulus, name="PoissonInput")

stimulus = obi.ConstantCurrentClampSomaticStimulus(
    timestamps=timestamps.ref, duration=2000.0, amplitude=0.5
)
sim_form.add(stimulus, name="CurrentClampInput")

# Recording
recording = obi.SomaVoltageRecording()
sim_form.add(recording, name="SomaVoltage")

# Initialization block
init = obi.MEModelWithSynapsesCircuitSimulationScanConfig.Initialize(
    circuit=obi.MEModelWithSynapsesCircuitFromID(id_str=entity_ID),
    simulation_length=sim_duration,
)
sim_form.set(init, name="initialize")
# Validated Config
validated_sim_conf = sim_form.validated_config()

print(validated_sim_conf)

# === 2. Wrap into a Simulation ===
grid_scan = obi.GridScanGenerationTask(form=validated_sim_conf, coordinate_directory_option="ZERO_INDEX", output_root='../../../obi-output/run_circuit_simulations/grid_scan')
grid_scan.execute(db_client=db_client)
obi.run_tasks_for_generated_scan(grid_scan, db_client=db_client)

In [ ]:
simulation_config_path = grid_scan.single_configs[0].coordinate_output_root / "simulation_config.json"
print(simulation_config_path)

circuit_folder = grid_scan.single_configs[0].coordinate_output_root / "sonata_circuit"

In [ ]:
# Remove the old compiled mod files folder
! rm -r arm64/
# flag DISABLE_REPORTINGLIB to skip SonataReportHelper.mod and SonataReport.mod from compilation.
!../../.venv/bin/nrnivmodl -incflags "-DDISABLE_REPORTINGLIB" {circuit_folder}/mod

In [ ]:
# === 4. Run the simulation (BlueCelluLab backend) ===
from obi_one.scientific.library.simulation_execution import run

run(
    simulation_config=simulation_config_path,
    simulator='bluecellulab',
    save_nwb=False
)

## Results
The results are stored in the `output` directory. You can analyze the voltage traces and other outputs as needed.

In [ ]:
import bluepysnap
snap_simulation = bluepysnap.Simulation(simulation_config_path)
spikes = snap_simulation.spikes
print(
    spikes.time_start,
    spikes.time_stop,
    spikes.dt
)
print(spikes.population_names)

In [ ]:
spike_pop = spikes[spikes.population_names[0]]
node_population = spike_pop.nodes
filtered = spikes.filter( t_start=spikes.time_start, t_stop=spikes.time_stop)
filtered.report.head()

In [ ]:
snap_simulation.reports

In [ ]:
soma_report = snap_simulation.reports['SomaVoltage']
print(
    soma_report.time_start,
    soma_report.time_stop,
    soma_report.dt
)  # Gives a warning in case the dt differs from simulation.dt